# MPFST EEG demo on PhysioNet excerpt
This notebook loads the small PhysioNet ds006036 excerpt shipped in `data/` and runs the end-to-end MPFST coherence + avalanche + Jacobian pipeline.

## Data prerequisites
This walkthrough expects `data/flip_example1_vlPFC_concat.npy`, which ships with the repository as a small Dryad-derived excerpt. 
If you removed it (or cloned with Git LFS filters that skipped large files), regenerate it with:

```bash
python scripts/convert_dryad_lfp_to_npy.py \
  --mat data/doi_10_5061_dryad_9w0vt4bnp__v20251104/data.mat \
  --field data.example1_vlPFC_lfp \
  --mode concat \
  --out-npy data/flip_example1_vlPFC_concat.npy
```

Make sure you have also run `pip install -e .` from the repo root so the `mpfst_eeg_jacobian` package can be imported inside this notebook.

In [ ]:
%matplotlib inline
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from mpfst_eeg_jacobian.pipeline import run_eeg_to_jacobian_avalanches
from mpfst_eeg_jacobian.plotting import plot_meter_and_valve, plot_avalanche_ccdf

In [ ]:
data_path = Path("data/flip_example1_vlPFC_concat.npy")
fs = 1000.0
channels = list(range(4))
latent_bands = ["theta", "alpha", "beta", "gamma"]

if not data_path.exists():
    raise FileNotFoundError(
        "Missing data/flip_example1_vlPFC_concat.npy. Run scripts/convert_dryad_lfp_to_npy.py "
        "to regenerate it before executing this notebook."
    )

eeg = np.load(data_path)
results = run_eeg_to_jacobian_avalanches(
    eeg=eeg,
    fs=fs,
    channel_indices=channels,
    band="beta",
    latent_bands=latent_bands,
    gate_q1=0.20,
    gate_q2=0.60,
    valve_quantile=0.35,
    jacobian_ridge=1e-4,
 )

In [ ]:
coh = results["coherence"]
jac = results["jacobian_metrics"]
tail = coh.tail_fit

print(f"Windows: {len(coh.times)}")
print(f"mu median    : {np.median(coh.mu):.3f}")
print(f"gamma median : {np.median(coh.gamma):.3f}")
print(f"H median     : {np.median(coh.H):.3f}")
print(f"m_l median   : {np.median(coh.m_l):.3f}")
print(f"Valve quantile: 0.35 (default)")
print("Tail fit:", tail)
print("Jacobian metrics:", jac)

In [ ]:
fig1 = plot_meter_and_valve(coh.times, coh.m_l, coh.m_hat, coh.valve)
fig1.suptitle("Meter and valve dynamics")
plt.show()

sizes = coh.avalanches["size"]
fig2 = plot_avalanche_ccdf(sizes, xmin=tail.get("xmin"))
fig2.suptitle("Avalanche CCDF")
plt.show()